In [2]:
from ultralytics import YOLO
import math

from deep_sort.application_util import preprocessing
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections as gdet

import cv2
import glob
from pathlib import Path

# own
from utils.utils import (get_frames, SEQ_01, SEQ_02, SEQ_03)
from utils.deepsort_utils import LABELS_DICT, UNKNOWN_DEFAULT

In [ ]:
deepsort_model_ = 'path/to/deepsort_model' # TODO missing
yolo_model_ = 'path/to/yolo_model' # TODO missing

encoder = gdet.create_box_encoder(deepsort_model_, batch_size=1)
_metric = nn_matching.NearestNeighborDistanceMetric("cosine", 0.4, None)
tracker = Tracker(_metric, n_init=0)
detector = YOLO(yolo_model_)

In [ ]:
frames = [get_frames(SEQ_01, frame_num_=i) for i in range(10)] #TODO: implement all_frames functionality

for _frame in frames:
    ...